# Building our Input Pipeline

In [1]:
import numpy as np 

In [5]:
with open('movie-xids.npy','rb') as f:
    Xids = np.load(f, allow_pickle=True)
with open('movie-xmasks.npy','rb') as f:
    Xmask = np.load(f, allow_pickle=True)
with open('movie-labels.npy','rb') as f:
    labels = np.load(f, allow_pickle=True)

In [7]:

Xids.shape

(156060, 512)

In [8]:
labels

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [9]:
import tensorflow as tf

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

Metal device set to: Apple M1


2022-11-14 18:52:50.680835: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-14 18:52:50.681995: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.int64, name=None), TensorSpec(shape=(512,), dtype=tf.int64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [15]:
def map_func(input_ids,masks,labels):
    return{'input_ids': input_ids,
          'attention_mask':masks},labels

In [16]:
dataset = dataset.map(map_func)

In [17]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int64, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [18]:
batch_size = 16

In [20]:
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

In [21]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [22]:
split = 0.9 

In [23]:
size = int((Xids.shape[0] / batch_size)*split)

In [24]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [25]:
tf.data.experimental.save(train_ds,'train')
tf.data.experimental.save(val_ds,'val')

Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


2022-11-14 19:06:42.310775: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [26]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [27]:
ds = tf.data.experimental.load('train',element_spec= train_ds.element_spec)

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.
